In [99]:
import os

In [100]:
os.listdir()

['Analyzing Final Results.ipynb',
 'FInal models.ipynb',
 'metrics_final_individual_content.json',
 'metrics_final_individual_determinant.json',
 'metrics_final_macro_content.json',
 'metrics_final_macro_determinants.json',
 'nancy_contents_individual_labels_eng.csv',
 'nancy_contents_macro_labels_english.csv',
 'nancy_determinants_individual_labels_eng.csv',
 'nancy_determinants_macro_labels_english.csv',
 'TO-DO.txt']

In [101]:
final_results = ['metrics_final_individual_content.json',
 'metrics_final_individual_determinant.json',
 'metrics_final_macro_content.json',
 'metrics_final_macro_determinants.json']

In [102]:
import pandas as pd
import json

In [103]:
# with open("metrics_final_individual_content.json", "r") as f:
#     data = json.load(f)

# validation_metrics = data[0]["metrics"]["best_metrics"]
# validation_metrics.keys()

# del validation_metrics["proba_outputs"]
# del validation_metrics["actual_classes"]

# test = data[0]["metrics"]["test"]
# test.keys()

# del test["test_proba"]
# del test["test_targets"]

# df  = pd.DataFrame(test, index=[0])
# list(data[0]["metrics"].keys())[-2]

In [104]:
# list(data[0]["metrics"].keys())[-2]

In [105]:
# data[0]["metrics"]["epoch_31"]

In [106]:
# list(data[0]["metrics"]["epoch_5"].keys())[-2:]

In [107]:
# data[0]["metrics"]["test"].keys()

In [108]:
import numpy as np

In [109]:
from sklearn.metrics import hamming_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

In [110]:
final_results

['metrics_final_individual_content.json',
 'metrics_final_individual_determinant.json',
 'metrics_final_macro_content.json',
 'metrics_final_macro_determinants.json']

In [111]:
# with open("metrics_final_individual_determinant.json", "r") as f:
#     data = json.load(f)

In [112]:
# data[0]["metrics"]["test"]["test_proba"]

In [130]:
results_test = pd.DataFrame()
results_validation = pd.DataFrame()

for model in final_results:
    model_name = model.split(".json")[0]
    with open(model, "r") as f:
        data = json.load(f)
    
    last_epoch = list(data[0]["metrics"].keys())[-3]
    validation_metrics = data[0]["metrics"][last_epoch]
    params = data[0]["hyperparameters"]
    # print(list(data[0]["metrics"].keys()))
    
    y_true_test = data[0]["metrics"]["test"]["test_targets"]
    prob_test = data[0]["metrics"]["test"]["test_proba"]
    
    test_metrics = data[0]["metrics"]["test"]
    
    list_keys = list(validation_metrics.keys())[-2:]
    
    argmaxed_predictions = []
    
    for keys in list(validation_metrics.keys()):
        if keys in list_keys[-2]:
            below_threshold = validation_metrics[keys]
            for pred in below_threshold:
                for pre in pred:
                    # print(pre)
                    pre_array = np.array(pre)
                    y_pred = np.zeros(pre_array.shape)
                    y_pred[np.where(pre_array>=0.5)] = 1
                    # print(y_pred)
                    
                    filter_ = [1 for element in pre if element > 0.5]
                    positive_preds = [pre.index(element) for element in pre if element > 0.5]
                    # print(filter_)
                    if 1 not in filter_:
                        idx = np.argmax(pre)
                        best_pred = pre[idx]
                        # print(best_pred)
                        y_pred[idx] = 1
                        
                    argmaxed_predictions.append(y_pred)
                    
            del validation_metrics[keys]
        elif keys in list_keys[-1]:
            # print(validation_metrics[keys])
            y_true = validation_metrics[keys]
            # del validation_metrics[keys]
            
    argmaxed_predictions = np.array(argmaxed_predictions).astype(np.float32)
    y_true = np.array(y_true).astype(np.float32)            
    
    # now i have the argmaxed preds and the true preds
    
    f1 = f1_score(y_true, argmaxed_predictions, average="micro")
    acc = accuracy_score(y_true, argmaxed_predictions)
    precision = precision_score(y_true, argmaxed_predictions, average="micro")
    recall = recall_score(y_true, argmaxed_predictions, average="micro")
    # report =classification_report(y_true, argmaxed_predictions, labels=data[0]["metrics"]["labels"])
    hamming = hamming_loss(y_true, argmaxed_predictions)

    # print(report)

    for key in list(validation_metrics.keys()):
        new_key_name = key.split("_epoch")[0]
        validation_metrics[new_key_name] = validation_metrics[key]
        # del validation_metrics[key]
    
    val_arg = {}
    val_arg["model"] = model_name
    val_arg["f1_arg"] = f1
    val_arg["acc_arg"] = acc
    val_arg["precision_arg"] = precision
    val_arg["recall_arg"] = recall
    
    
    
    test_metrics["model"] = model_name
    # print(test_metrics.keys())
    
    # prob_test = data[0]["metrics"]["test"]["test_proba"]
    # print(prob_test)
    # y_true_test = data[0]["metrics"]["test"]["test_targets"]
    
    argmaxed_predictions_test = []
    for pred in prob_test:
                for pre in pred:
                    # print(pre)
                    pre_array = np.array(pre)
                    y_pred_test = np.zeros(pre_array.shape)
                    y_pred_test[np.where(pre_array>=0.5)] = 1
                    #print(y_pred)
                    
                    filter_ = [1 for element in pre if element > 0.5]
                    positive_preds = [pre.index(element) for element in pre if element > 0.5]
                    # print(filter_)
                    if 1 not in filter_:
                        idx = np.argmax(pre)
                        best_pred = pre[idx]
                        # print(best_pred)
                        # print(y_pred)
                        y_pred_test[idx] = 1
                        
                    argmaxed_predictions_test.append(y_pred_test)
    
    argmaxed_predictions_test = np.array(argmaxed_predictions_test)
    y_true_test = np.array(y_true_test)
    
    f1_test = f1_score(y_true_test, argmaxed_predictions_test, average="micro")
    acc_test = accuracy_score(y_true_test, argmaxed_predictions_test)
    precision_test = precision_score(y_true_test, argmaxed_predictions_test, average="micro")
    recall_test = recall_score(y_true_test, argmaxed_predictions_test, average="micro")
    hamming_test = hamming_loss(y_true_test, argmaxed_predictions_test)
    # report_test = classification_report(y_true_test, argmaxed_predictions_test, labels=data[0]["metrics"]["labels"])
    # print(report_test)
    
    test_arg = {}
    test_arg["model"] = model_name
    test_arg["f1_arg"] = f1_test
    test_arg["acc_arg"] = acc_test
    test_arg["precision_arg"] = precision_test
    test_arg["recall_arg"] = recall_test
    test_arg["hamming"] = hamming_test
    # del test_metrics["test_proba"]
    # del test_metrics["test_targets"]


    df_val = pd.DataFrame(val_arg, index=[0])
    df_test = pd.DataFrame(test_arg, index=[0])
    df_params = pd.DataFrame(params, index=[0])

    results_test = pd.concat([results_test, df_test], axis=0)
    results_validation = pd.concat([results_validation, df_val], axis=0)
    
    print(model_name)
    print(data[0]["metrics"]["labels"])
    print(len(data[0]["metrics"]["labels"]))

metrics_final_individual_content
['(A)', '(AD)', '(H)', '(HD)', 'A', 'ABS', 'AD', 'ALIM', 'ANAT', 'ARCH', 'ART', 'BOT', 'ELEM', 'FRAG', 'GÉO', 'H', 'HD', 'MQ', 'NAT', 'OBJ', 'PAYS', 'RADIO', 'SC', 'SCÈNE', 'SEX', 'SG', 'VÊT']
27
metrics_final_individual_determinant
['C', "C'", "C'F", 'CF', "CF'", 'CLOB', 'CLOBF', 'E', 'EF', 'F', 'FC', "FC'", 'FCLOB', 'FE', 'K', 'KAN', 'KOB', 'KP']
18
metrics_final_macro_content
['animal_sum', 'human_sum', 'abs_sum', 'food_sum', 'art_arch_sum', 'nature_sum', 'fragment_sum', 'geo_sum', 'object_sum', 'science_sum', 'graphic_sum']
11
metrics_final_macro_determinants
['color_sum', 'threat_sum', 'fading_sum', 'form_sum', 'kinesthetics_sum']
5


In [128]:
results_test

,model,f1_arg,acc_arg,precision_arg,recall_arg,hamming
0,metrics_final_individual_content,0.816327,0.739130,0.800000,0.833333,0.014493
0,metrics_final_individual_determinant,0.465116,0.400000,0.500000,0.434783,0.063889
0,metrics_final_macro_content,0.888889,0.863636,0.909091,0.869565,0.020661
0,metrics_final_macro_determinants,0.590909,0.444444,0.619048,0.565217,0.200000


In [129]:
print(results_test.to_latex())

\begin{tabular}{llrrrrr}
\toprule
 & model & f1_arg & acc_arg & precision_arg & recall_arg & hamming \\
\midrule
0 & metrics_final_individual_content & 0.816327 & 0.739130 & 0.800000 & 0.833333 & 0.014493 \\
0 & metrics_final_individual_determinant & 0.465116 & 0.400000 & 0.500000 & 0.434783 & 0.063889 \\
0 & metrics_final_macro_content & 0.888889 & 0.863636 & 0.909091 & 0.869565 & 0.020661 \\
0 & metrics_final_macro_determinants & 0.590909 & 0.444444 & 0.619048 & 0.565217 & 0.200000 \\
\bottomrule
\end{tabular}



print(results_test.to_latex())